<a href="https://colab.research.google.com/github/tarakantaacharya/NLPinternal/blob/main/rag_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
! pip install langchain
! pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [192]:
! pip install chromadb

In [ ]:
! pip install faiss-cpu

In [ ]:
! pip install scann

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 25.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df_ap = pd.read_csv('/content/ap_eapcet-last-rank-2023.csv')
df_ts = pd.read_csv('/content/TSEAMCET_2023_FINALPHASE.csv')

In [ ]:
df_ap['State'] = "Andhra Pradesh"
df_ts['State'] = "Telangana"

In [ ]:
df_ts.columns

Index(['Inst\n Code', 'Institute Name', 'Place', 'Dist \nCode', 'Co Education',
       'College Type', 'Year of Estab', 'Branch Code', 'Branch Name',
       'OC \nBOYS', 'OC \nGIRLS', 'BC_A \nBOYS', 'BC_A \nGIRLS', 'BC_B \nBOYS',
       'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS', 'BC_D \nBOYS',
       'BC_D \nGIRLS', 'BC_E \nBOYS', 'BC_E \nGIRLS', 'SC \nBOYS',
       'SC \nGIRLS', 'ST \nBOYS', 'ST \nGIRLS', 'EWS \nGEN OU',
       'EWS \nGIRLS OU', 'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.columns

Index(['SNO', 'INSTCODE', 'NAME OF THE INSTITUTION', 'TYPE', 'INST\r_RE\rG',
       'DIST', 'PLACE', 'COED', 'AFFL.', 'ESTD', 'A_RE\rG', 'branch_\rcode',
       'OC_BO\rYS', 'OC_GIR\rLS', 'SC_BO\rYS', 'SC_GIR\rLS', 'ST_BO\rYS',
       'ST_GIR\rLS', 'BCA_B\rOYS', 'BCA_GI\rRLS', 'BCB_B\rOYS', 'BCB_GI\rRLS',
       'BCC_B\rOYS', 'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BCE_B\rOYS',
       'BCE_GI\rRLS', 'OC_EWS_B\rOYS', 'OC_EWS_G\rIRLS', 'Unnamed: 30',
       'State'],
      dtype='object')

In [ ]:
df_ts.rename(columns={
    'Inst\n Code': 'Institute Code',
    'Institute Name': 'Institute Name',
    'College Type': 'College Type',
    'Place': 'Location',
    'Dist \nCode': 'District Code',
    'Co Education': 'Co-Ed Status',
    'Year of Estab': 'Established Year',
    'Branch Code': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Tuition Fee': 'Tuition Fee',
    'Affiliated To': 'Affiliated To',
    'EWS \nGEN OU': 'EWS Boys',
    'EWS \nGIRLS OU': 'EWS Girls',
    'OC \nBOYS': 'OC Boys',
    'OC \nGIRLS': 'OC Girls',
    'SC \nBOYS': 'SC Boys',
    'SC \nGIRLS': 'SC Girls',
    'ST \nBOYS': 'ST Boys',
    'ST \nGIRLS': 'ST Girls',
    'BC_A \nBOYS': 'BC-A Boys',
    'BC_A \nGIRLS': 'BC-A Girls',
    'BC_E \nBOYS': 'BC-E Boys',
    'BC_E \nGIRLS': 'BC-E Girls'
}, inplace=True)

In [ ]:
df_ts.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS',
       'BC_D \nBOYS', 'BC_D \nGIRLS', 'BC-E Boys', 'BC-E Girls', 'SC Boys',
       'SC Girls', 'ST Boys', 'ST Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.rename(columns={
    'INSTCODE': 'Institute Code',
    'NAME OF THE INSTITUTION': 'Institute Name',
    'TYPE': 'College Type',
    'PLACE': 'Location',
    'DIST': 'District Code',
    'COED': 'Co-Ed Status',
    'ESTD': 'Established Year',
    'branch_\rcode': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Unnamed: 30': 'Tuition Fee',
    'AFFL.': 'Affiliated To',
    'OC_EWS_B\rOYS': 'EWS Boys',
    'OC_EWS_G\rIRLS': 'EWS Girls',
    'BCA_B\rOYS': 'BC-A Boys',
    'BCA_GI\rRLS': 'BC-A Girls',
    'BCE_B\rOYS': 'BC-E Boys',
    'BCE_GI\rRLS': 'BC-E Girls',
    'OC_BO\rYS': 'OC Boys',
    'OC_GIR\rLS': 'OC Girls',
    'SC_BO\rYS': 'SC Boys',
    'SC_GIR\rLS': 'SC Girls',
    'ST_BO\rYS': 'ST Boys',
    'ST_GIR\rLS': 'ST Girls',
}, inplace=True)

In [ ]:
df_ap.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'INST\r_RE\rG', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS',
       'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BC-E Boys', 'BC-E Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts["Institution Registered"] = "AU"

In [ ]:
df_ap.rename(columns={"INST\r_RE\rG" : "Institution Registered"}, inplace=True)

In [ ]:
df_ap_new = df_ap.drop(columns=('A_RE\rG'))

In [ ]:
df_ts_new = df_ts.drop(columns=['BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS','BC_D \nBOYS', 'BC_D \nGIRLS'])

In [ ]:
df_ap_new = df_ap.drop(columns=['BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS','BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS'])

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys',
       'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=['A_RE\rG'], inplace=True)

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=["SNO"], inplace=True)

In [ ]:
df_ap_new.columns

Index(['Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
import pandas as pd
common_cols = list(set(df_ap_new.columns) & set(df_ts_new.columns))
df_combined = pd.concat([df_ap_new[common_cols], df_ts_new[common_cols]], ignore_index=True)

In [ ]:
df_combined.columns

Index(['Tuition Fee', 'OC Girls', 'SC Boys', 'Established Year',
       'Institute Code', 'State', 'Affiliated To', 'BC-E Girls',
       'College Type', 'Co-Ed Status', 'BC-A Boys', 'BC-A Girls', 'OC Boys',
       'SC Girls', 'Location', 'Branch Code', 'Institution Registered',
       'ST Boys', 'Institute Name', 'BC-E Boys', 'EWS Boys', 'District Code',
       'ST Girls', 'EWS Girls'],
      dtype='object')

In [ ]:
len(df_ap_new)

1572

In [ ]:
len(df_ts_new)

958

In [ ]:
len(df_combined)

2530

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
Tuition Fee,59
OC Girls,1
SC Boys,177
Established Year,0
Institute Code,0
State,0
Affiliated To,0
BC-E Girls,0
College Type,0
Co-Ed Status,0


In [ ]:
df_combined = df_combined.drop(columns=['BC-A Boys', 'BC-A Girls','SC Girls','ST Boys','BC-E Boys', 'EWS Boys', 'ST Girls','EWS Girls'])

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
Tuition Fee,59
OC Girls,1
SC Boys,177
Established Year,0
Institute Code,0
State,0
Affiliated To,0
BC-E Girls,0
College Type,0
Co-Ed Status,0


In [ ]:
df_combined.dropna(how='any', inplace=True)

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
Tuition Fee,0
OC Girls,0
SC Boys,0
Established Year,0
Institute Code,0
State,0
Affiliated To,0
BC-E Girls,0
College Type,0
Co-Ed Status,0


In [ ]:
df_combined = df_combined.drop(columns=['SC Boys','BC-E Girls'])

In [ ]:
df_combined.columns

Index(['Tuition Fee', 'OC Girls', 'Established Year', 'Institute Code',
       'State', 'Affiliated To', 'College Type', 'Co-Ed Status', 'OC Boys',
       'Location', 'Branch Code', 'Institution Registered', 'Institute Name',
       'District Code'],
      dtype='object')

In [ ]:
df_combined.rename(columns={
    'OC Boys': 'Best Boy Rank',
    'OC Girls': 'Best Girl Rank',
}, inplace=True)

In [ ]:
df_combined.columns

Index(['Tuition Fee', 'Best Girl Rank', 'Established Year', 'Institute Code',
       'State', 'Affiliated To', 'College Type', 'Co-Ed Status', 'Best Rank',
       'Location', 'Branch Code', 'Institution Registered', 'Institute Name',
       'District Code'],
      dtype='object')

In [ ]:
df_combined.to_csv('college_list.csv')

In [ ]:
len(df_combined)

2294

In [216]:
df_combined.iloc[1654]

,1800
Tuition Fee,60000.0
Best Girl Rank,35962
Established Year,2010
Institute Code,AARM
State,Telangana
Affiliated To,JNTUH
College Type,PVT
Co-Ed Status,COED
Best Rank,35962.0
Location,BANDLAGUDA


In [217]:
# Convert 'Best Rank' column to numeric before comparison
df_combined['Best Rank'] = pd.to_numeric(df_combined['Best Rank'], errors='coerce')

# Now you can perform the comparison
if (df_combined['Best Rank'] < 5000).any():  # Use .any() to check if any value in the Series satisfies the condition
    # Since the condition applies to the entire Series, use boolean indexing to select the relevant rows
    filtered_df = df_combined[df_combined['Best Rank'] < 5000]
    print(filtered_df['Institute Name'])

96                        JNTUK COLLEGE OF ENGG. KAKINADA
97                        JNTUK COLLEGE OF ENGG. KAKINADA
705                   A U COLLEGE OF ENGG.\rVISAKHAPATNAM
706                   A U COLLEGE OF ENGG.\rVISAKHAPATNAM
760         GAYATHRI VIDYA PARISHAD COLL. OF\rENGINEERING
761         GAYATHRI VIDYA PARISHAD COLL. OF\rENGINEERING
1572    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1573           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1574      V N R VIGNANA JYOTHI INSTITUTE OF ENGG AND TECH
1575    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1576           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1577                       O U COLLEGE OF ENGG  HYDERABAD
1578      V N R VIGNANA JYOTHI INSTITUTE OF ENGG AND TECH
1579                        VASAVI COLLEGE OF ENGINEERING
1580    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1581    JNTUH UNIVERSITY COLLEGE OF ENGG SCI AND TECH ...
1582           CHAITANYA BHARATHI INSTITUTE OF TECHNOLOGY
1583          

In [ ]:
import re
import groq  # Groq API for LLM

# Set up Groq API key (replace with your actual key)
groq.api_key = "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"  # Remove API key before sharing code publicly!

In [190]:
df_combined.columns

Index(['Tuition Fee', 'Best Girl Rank', 'Established Year', 'Institute Code',
       'State', 'Affiliated To', 'College Type', 'Co-Ed Status', 'Best Rank',
       'Location', 'Branch Code', 'Institution Registered', 'Institute Name',
       'District Code'],
      dtype='object')

In [191]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# Load your dataset (Modify this if using a database)
df = pd.read_csv("college_list.csv")

# Convert table rows into structured text
def row_to_text(row):
    return (
        f"Institute Name: {row['Institute Name']}\n"
        f"Institute Code: {row['Institute Code']}\n"
        f"Established Year: {row['Established Year']}\n"
        f"Affiliated To: {row['Affiliated To']}\n"
        f"College Type: {row['College Type']}\n"
        f"Co-Ed Status: {row['Co-Ed Status']}\n"
        f"Best Rank: {row['Best Rank']}\n"
        f"Best Girl Rank: {row['Best Girl Rank']}\n"
        f"Tuition Fee: {row['Tuition Fee']}\n"
        f"Location: {row['Location']}, {row['State']}, District: {row['District Code']}\n"
        f"Branch Code: {row['Branch Code']}\n"
        f"Institution Registered: {row['Institution Registered']}\n"
    )

# Convert all rows to text format
texts = df.apply(row_to_text, axis=1).tolist()

# Split large text chunks if needed
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = [chunk for text in texts for chunk in text_splitter.split_text(text)]

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings
embeddings = embedding_model.embed_documents(chunks)

print(f"✅ Converted {len(chunks)} text chunks from the table.")

✅ Converted 4459 text chunks from the table.


In [194]:
import chromadb

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="vector_db/college_chromadb")

# Create collection
collection = chroma_client.get_or_create_collection(name="colleges")

# Store data in ChromaDB
for idx, text in enumerate(texts):
    collection.add(
        ids=[str(idx)],  # Unique ID for each entry
        embeddings=[embeddings[idx]],  # Embedding vector
        metadatas=[{"text": text}]  # Metadata for retrieval
    )

print("✅ Embeddings stored in ChromaDB!")

✅ Embeddings stored in ChromaDB!


In [196]:
import faiss

# Initialize FAISS index
embeddings_np = np.array(embeddings, dtype=np.float32)

embedding_dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance
index.add(embeddings_np)

# Save FAISS index to a file within the directory
faiss.write_index(index, "vector_db/college_faiss_index.bin") # Changed to write to a .bin file instead of directory

print("✅ FAISS index created and saved!")

✅ FAISS index created and saved!


In [197]:
import os
from groq import Groq

# 🔹 Set your API key
os.environ["GROQ_API_KEY"] = "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"

# 🔹 Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [198]:
def retrieve_from_faiss(query, index, k=5):
    query_embedding = embedding_model.embed_query(query)
    query_embedding = np.array([query_embedding], dtype=np.float32)

    distances, indices = index.search(query_embedding, k)
    results = [texts[idx] for idx in indices[0] if idx < len(texts)]
    return results

In [204]:
def retrieve_from_chromadb(query, collection, k=5):
    query_embedding = embedding_model.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )
    # Get metadatas from the first document in results
    metadatas = results['documents'][0]
    return [metadata for metadata in metadatas] # Return the list of metadatas

In [221]:
def generate_answer(query, faiss_index, chroma_collection, k=5):
    # 🔹 Retrieve from FAISS
    faiss_results = retrieve_from_faiss(query, faiss_index, k)

    # 🔹 Retrieve from ChromaDB
    chroma_results = retrieve_from_chromadb(query, chroma_collection, k)

    # 🔹 Merge results (remove duplicates and None values)
    combined_results = list(set(result for result in faiss_results + chroma_results if result is not None)) # Filter out None values

    # 🔹 Create context from retrieved documents
    context = "\n".join(combined_results)

    # 🔹 Prepare prompt for LLM
    prompt = f"Use the stored data to answer the query and don't use any outside information:\n{context}\n\nQuery: {query}\nAnswer:"

    # 🔹 Call the LLM API (Groq or OpenAI)
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # Choose the best model
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    return response

In [225]:
# Load FAISS index
faiss_index = faiss.read_index("vector_db/college_faiss_index.bin")

# Load ChromaDB collection
chroma_client = chromadb.PersistentClient(path="vector_db/college_chromadb")
chroma_collection = chroma_client.get_or_create_collection(name="colleges")

# Example Query
query = "Best engineering colleges in State Andhra Pradesh under rank 5000"
response = generate_answer(query, faiss_index, chroma_collection)

print("🤖 Chatbot Response:")
print(response.choices[0].message.content)

🤖 Chatbot Response:
Based on the stored data, the best engineering colleges in State Andhra Pradesh under rank 5000 are:

1. ADITYA ENGINEERING COLLEGE (Institute Code: ADTP) with Best Rank: 22560.0


---

---